In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', 50)

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:/Users/anand/Desktop/shreyansh sir/ML725/Assignment1/train.csv')
df.head()

,instance (id),date,season,hour,holiday,weekday,working day,weather situation,temperature,atemperature,humidity,wind speed,count
0,0,2011-09-30,4,5,0,Friday,1,1,0.52,0.5000,0.83,0.1940,26
1,1,2011-06-03,2,11,0,Friday,1,1,0.64,0.6212,0.31,0.3284,192
2,2,2012-03-23,2,10,0,Friday,1,2,0.56,0.5303,0.88,0.1045,275
3,3,2012-12-30,1,18,0,Sunday,0,2,0.24,0.2121,3.00,0.2985,125
4,4,2012-02-07,1,15,0,Tuesday,1,1,0.44,0.4394,0.38,0.1940,147


In [47]:
def out(x):
    if ((x < df['wind speed'].mean() + 2 * df['wind speed'].std()) and (x > df['wind speed'].mean() - 2 * df['wind speed'].std())):
        return 0
    else:
        return 1

In [48]:
df['out'] = df['wind speed'].apply(lambda x: out(x))

In [49]:
df.out.value_counts()

0    12440
1     1425
Name: out, dtype: int64

In [50]:
df.drop(df[df['out'] == 1].index, inplace = True) 

In [51]:
df.shape

(12440, 14)

In [3]:
df.shape

(13865, 13)

In [27]:
df['weather situation'] = pd.Categorical(df['weather situation'], categories=[1,2,3,4])

In [28]:
df['weather situation'].unique()

[1, 2, 3]
Categories (3, int64): [1, 2, 3]

In [4]:
# dropping unnecessary features for now
df = df.drop(['instance (id)', 'date', 'temperature', 'working day' ], axis=1)
df.head()

,season,hour,holiday,weekday,weather situation,atemperature,humidity,wind speed,count
0,4,5,0,Friday,1,0.5000,0.83,0.1940,26
1,2,11,0,Friday,1,0.6212,0.31,0.3284,192
2,2,10,0,Friday,2,0.5303,0.88,0.1045,275
3,1,18,0,Sunday,2,0.2121,3.00,0.2985,125
4,1,15,0,Tuesday,1,0.4394,0.38,0.1940,147


In [5]:
# Creating dummies

df = pd.get_dummies(data=df, columns=[ 'weekday'], drop_first=True)
df.head()

,season,hour,holiday,weather situation,atemperature,humidity,wind speed,count,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,4,5,0,1,0.5000,0.83,0.1940,26,0,0,0,0,0,0
1,2,11,0,1,0.6212,0.31,0.3284,192,0,0,0,0,0,0
2,2,10,0,2,0.5303,0.88,0.1045,275,0,0,0,0,0,0
3,1,18,0,2,0.2121,3.00,0.2985,125,0,0,1,0,0,0
4,1,15,0,1,0.4394,0.38,0.1940,147,0,0,0,0,1,0


In [6]:
# scaling
import pandas as pd
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df[['wind speed','humidity']] = scaler.fit_transform(df[['wind speed','humidity']])

In [7]:
df.head()

,season,hour,holiday,weather situation,atemperature,humidity,wind speed,count,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,4,5,0,1,0.5000,0.276667,0.064667,26,0,0,0,0,0,0
1,2,11,0,1,0.6212,0.103333,0.109467,192,0,0,0,0,0,0
2,2,10,0,2,0.5303,0.293333,0.034833,275,0,0,0,0,0,0
3,1,18,0,2,0.2121,1.000000,0.099500,125,0,0,1,0,0,0
4,1,15,0,1,0.4394,0.126667,0.064667,147,0,0,0,0,1,0


In [8]:
print(df.shape)

(13865, 14)


In [9]:
print(df.shape)
y = df['count']
X = df.drop('count', axis=1)
print(y.shape)
print(X.shape)

(13865, 14)
(13865,)
(13865, 13)


In [10]:
df.columns

Index(['season', 'hour', 'holiday', 'weather situation', 'atemperature',
       'humidity', 'wind speed', 'count', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday'],
      dtype='object')

### Modelling

In [11]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
X.shape[1]

13

In [13]:
X.columns

Index(['season', 'hour', 'holiday', 'weather situation', 'atemperature',
       'humidity', 'wind speed', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday'],
      dtype='object')

In [15]:
model = Sequential()
model.add(Dense(1, input_dim=X.shape[1], 
                kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=42),
               bias_initializer='zeros'))
opt = SGD(lr=0.005) 
model.compile(loss='mean_squared_error', optimizer=opt)
model.fit(X, y, epochs=20000, batch_size=X.shape[0])

Epoch 1/20000
13865/13865 [==============================] - 0s 10us/step - loss: 63299.6484
Epoch 2/20000
13865/13865 [==============================] - 0s 1us/step - loss: 55910.0234
Epoch 3/20000
13865/13865 [==============================] - 0s 1us/step - loss: 50045.9609
Epoch 4/20000
13865/13865 [==============================] - 0s 1us/step - loss: 45390.4922
Epoch 5/20000
13865/13865 [==============================] - 0s 1us/step - loss: 41692.0898
Epoch 6/20000
13865/13865 [==============================] - 0s 1us/step - loss: 38751.9570
Epoch 7/20000
13865/13865 [==============================] - 0s 1us/step - loss: 36412.5781
Epoch 8/20000
13865/13865 [==============================] - 0s 1us/step - loss: 34549.4102
Epoch 9/20000
13865/13865 [==============================] - 0s 1us/step - loss: 33063.7188
Epoch 10/20000
13865/13865 [==============================] - 0s 1us/step - loss: 31877.3691
Epoch 11/20000
13865/13865 [==============================] - 0s 1us/step - lo

Epoch 90/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26233.1230
Epoch 91/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26227.4316
Epoch 92/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26221.7871
Epoch 93/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26216.2090
Epoch 94/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26210.6855
Epoch 95/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26205.2070
Epoch 96/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26199.7812
Epoch 97/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26194.4082
Epoch 98/20000
13865/13865 [==============================] - 0s 2us/step - loss: 26189.0859
Epoch 99/20000
13865/13865 [==============================] - 0s 1us/step - loss: 26183.8008
Epoch 100/20000
13865/13865 [==============================] - 0s 1us/

Epoch 178/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25868.7910
Epoch 179/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25865.7070
Epoch 180/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25862.6445
Epoch 181/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25859.5938
Epoch 182/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25856.5605
Epoch 183/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25853.5527
Epoch 184/20000
13865/13865 [==============================] - 0s 2us/step - loss: 25850.5449
Epoch 185/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25847.5664
Epoch 186/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25844.5938
Epoch 187/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25841.6406
Epoch 188/20000
13865/13865 [==============================]

13865/13865 [==============================] - 0s 1us/step - loss: 25646.7871
Epoch 266/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25644.6367
Epoch 267/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25642.4902
Epoch 268/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25640.3555
Epoch 269/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25638.2246
Epoch 270/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25636.1035
Epoch 271/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25633.9902
Epoch 272/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25631.8828
Epoch 273/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25629.7793
Epoch 274/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25627.6758
Epoch 275/20000
13865/13865 [==============================] - 0s 1us/step -

Epoch 353/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25477.8516
Epoch 354/20000
13865/13865 [==============================] - 0s 2us/step - loss: 25476.1152
Epoch 355/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25474.3809
Epoch 356/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25472.6602
Epoch 357/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25470.9316
Epoch 358/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25469.2070
Epoch 359/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25467.4902
Epoch 360/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25465.7734
Epoch 361/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25464.0605
Epoch 362/20000
13865/13865 [==============================] - 0s 1us/step - loss: 25462.3496
Epoch 363/20000
13865/13865 [==============================]

...

Epoch 11217/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22983.0000
Epoch 11218/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9980
Epoch 11219/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9922
Epoch 11220/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9824
Epoch 11221/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9707
Epoch 11222/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9648
Epoch 11223/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9609
Epoch 11224/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9512
Epoch 11225/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.9395
Epoch 11226/20000
13865/13865 [==============================] - 0s 2us/step - loss: 22982.9375
Epoch 11227/20000
13865/13865 [=========

Epoch 11303/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3926
Epoch 11304/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3848
Epoch 11305/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3750
Epoch 11306/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3691
Epoch 11307/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3652
Epoch 11308/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3516
Epoch 11309/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3477
Epoch 11310/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3477
Epoch 11311/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3340
Epoch 11312/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22982.3262
Epoch 11313/20000
13865/13865 [=========

Epoch 11389/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.8047
Epoch 11390/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7969
Epoch 11391/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7891
Epoch 11392/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7832
Epoch 11393/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7754
Epoch 11394/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7695
Epoch 11395/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7598
Epoch 11396/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7539
Epoch 11397/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7461
Epoch 11398/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.7402
Epoch 11399/20000
13865/13865 [=========

Epoch 11475/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2344
Epoch 11476/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2246
Epoch 11477/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2207
Epoch 11478/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2168
Epoch 11479/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2070
Epoch 11480/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.2051
Epoch 11481/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.1934
Epoch 11482/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.1816
Epoch 11483/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.1836
Epoch 11484/20000
13865/13865 [==============================] - 0s 1us/step - loss: 22981.1719
Epoch 11485/20000
13865/13865 [=========

KeyboardInterrupt: 

In [23]:
score = model.evaluate(xtest, ytest, batch_size=X.shape[0])
score

2773/2773 [==============================] - 0s 13us/step


12574.2646484375

In [ ]:
16934.1699
16934.1699
16824.3555

In [24]:
score = model.evaluate(xtrain, ytrain, batch_size=X.shape[0])
score

11092/11092 [==============================] - 0s 1us/step


12463.1953125

### Prediction

In [ ]:
# p = model.predict(xtrain)


In [42]:
prediction = model.predict(df)
print(prediction)

[[490.77625]
 [ 69.09042]
 [307.34134]
 ...
 [336.67926]
 [387.15802]
 [ 64.99655]]


In [43]:
count=0
for i in prediction:
    if i<0:
        count+=1
        print(i)
print(count)

[-68.13062]
[-0.5127716]
[-45.59014]
[-21.286095]
[-68.84442]
[-13.095764]
[-39.98863]
[-39.2097]
[-19.71817]
[-77.71853]
[-16.308105]
[-31.879162]
[-101.84255]
[-69.67408]
[-37.775684]
[-84.94196]
[-53.19934]
[-24.391739]
[-59.005146]
[-3.1457672]
[-4.9462433]
[-25.930206]
[-69.60642]
[-16.249634]
[-54.43273]
[-9.303406]
[-101.86566]
[-49.896767]
[-15.946854]
[-86.27037]
[-134.70885]
[-39.390297]
[-22.474457]
[-2.6689987]
[-33.055977]
[-41.25673]
[-65.5499]
[-8.289482]
[-2.9310684]
[-151.5521]
[-49.547363]
[-20.619644]
[-25.262615]
[-27.110725]
[-9.681328]
[-4.5842133]
[-68.537094]
[-43.797222]
[-23.019882]
[-7.1781387]
[-59.01848]
[-33.121426]
[-82.139984]
[-70.70196]
[-8.219795]
[-40.961124]
[-4.148308]
[-65.76308]
[-94.538284]
[-42.285175]
[-22.094273]
[-13.183609]
[-70.0578]
[-43.5362]
[-77.629524]
[-1.5100708]
[-28.873955]
[-85.78613]
[-2.6685028]
[-1.7858124]
[-56.563435]
[-10.577316]
[-5.088318]
[-82.258934]
[-27.368027]
[-75.87074]
[-31.50631]
[-81.65205]
[-50.430202]
[-108.77

In [36]:
t = pd.DataFrame(prediction, columns=['count']).reset_index(drop=False)
t.columns = ['instance (id)', 'count']
t.head()

,instance (id),count
0,0,490.777313
1,1,69.089035
2,2,307.340393
3,3,339.009460
4,4,126.622009


In [44]:
t['count'] = t['count'].astype('int')
t['count'] = t['count'].apply(lambda x : 0 if x < 0 else x)
t.to_csv('out1.csv', index=False)


In [45]:
t['count'].min()

0